## Make list of EPG - profiles and interfaces and deployed VLANS

In [215]:
import json
import re
import pandas as pd
import requests



admin = 'admin'
passwd = 'cisco!123'
apicip = '10.122.189.179'

get_block_url = 'https://' + apicip + '/api/policymgr/mo/uni.json?query-target=subtree&target-subtree-class=infraPortBlk'
get_switch_block_url = 'https://' + apicip + '/api/policymgr/mo/uni.json?query-target=subtree&target-subtree-class=infraNodeBlk'
get_switch_profile_pg = 'https://' + apicip + '/api/policymgr/mo/uni.json?query-target=subtree&target-subtree-class=infraRsAccBaseGrp'
get_epg_and_path_url =  'https://' + apicip + '/api/class/fvAEPg.json?query-target=subtree&target-subtree-class=fvRsPathAtt'


def get_token():
   url = 'https://'+apicip+'/api/aaaLogin.json'

   payload = {
      "aaaUser": {
         "attributes": {
            "name": admin,
            "pwd": passwd
         }
      }
   }

   headers = {
      "Content-Type" : "application/json"
   }
   requests.packages.urllib3.disable_warnings()
   response = requests.post(url,data=json.dumps(payload), headers=headers, verify=False).json()

   token = response['imdata'][0]['aaaLogin']['attributes']['token']
   return token


def get_data(url):
   token = get_token()



   headers = {
      "Cookie" : f"APIC-Cookie={token}",
   }

   requests.packages.urllib3.disable_warnings()
   response = requests.get(url, headers=headers, verify=False)

   return response


# Get switch profile  with Ports

response = get_data(get_block_url).json()


pgs = response['imdata']
switch_profile_df = pd.DataFrame(columns=['switch_profile','fromPort','toPort'])
for pg in pgs:
#print(f\"fault name: {fault['faultInst']['created']['descr']['origSeverity']}\")
    name = pg['infraPortBlk']['attributes']['dn']
    fromPort = pg['infraPortBlk']['attributes']['fromPort']
    toPort = pg['infraPortBlk']['attributes']['toPort']
    
    
    p1 = re.compile(r'^uni\/infra\/s?p?accportprof\-(?P<name>\S*)\/s?h')
    m = p1.match(name)
    group = m.groupdict()

    pggs = group['name']
    switch_profile_df = switch_profile_df.append({'switch_profile': pggs, 'fromPort': fromPort,'toPort': toPort},ignore_index=True)

    
# Get Node Blocks

response = get_data(get_switch_block_url).json()


pgs = response['imdata']
node_blocks_df = pd.DataFrame(columns=['profile','fromNode','toNode'])
for pg in pgs:
    
    name = pg['infraNodeBlk']['attributes']['dn']
    fromNode = pg['infraNodeBlk']['attributes']['from_']
    toNode = pg['infraNodeBlk']['attributes']['to_']
    
    p1 = re.compile(r'^uni\/infra\/s?[np]prof\-(?P<name>\S*)\/[ls]')
    p2  = re.compile(r'^uni\/infra\/mgmtnodegrp\-(?P<name>\S*)\/n')
    
    m = p1.match(name)
    if m:
        group = m.groupdict()
    else:
         m = p2.match(name)
    
    pggs = group['name']
    node_blocks_df = s_df.append({'profile': pggs ,'fromNode': fromNode ,'toNode': toNode},ignore_index=True)
    
    

# Get Switch Profile and Policy Groups
    
response = get_data(get_switch_profile_pg ).json()


pgs = response['imdata']

policy_df = pd.DataFrame(columns=['switch_profile','policy_group'])
for pg in pgs:
    
    name = pg['infraRsAccBaseGrp']['attributes']['dn']
    tname = pg['infraRsAccBaseGrp']['attributes']['tDn']

   
    p1 = re.compile(r'^uni\/infra\/accportprof\-(?P<name>\S*)\/h')
    m1 = p1.match(name)
    group = m1.groupdict()
    switch_profile = group['name']
    
    
    
#    print (name)

    p1 = re.compile(r'^uni\/infra\/funcprof\/accbundle\-(?P<name>\S*)')
    p2 = re.compile(r'^uni\/infra\/funcprof\/f?c?accportgrp\-(?P<name>\S*)')

    m1 = p1.match(tname)
    m2 = p2.match(tname)

    print(tname)
    if m1:
        group = m1.groupdict()
        pggs = group['name']
        
    elif m2:
        group = m2.groupdict()
        pggs = group['name']
    
    

    policy_df = policy_df.append({'switch_profile': switch_profile ,'policy_group': pggs},ignore_index=True)
    
    
    
# Get EPG, encap and path

response = get_data(get_epg_and_path_url).json()

pgs = response['imdata']
epg_df = pd.DataFrame(columns=['tenant','ap','epg','pod','nodes','type','policy_group','encap'])

for pg in pgs:
    
    dn = pg['fvRsPathAtt']['attributes']['dn']
    encap = pg['fvRsPathAtt']['attributes']['encap']
   
    
    #plain port rdn
    p1 = re.compile(r'^uni\/tn\-(?P<tenant>\S*)\/ap-(?P<ap>\S*)\/epg\-(?P<epg>\S*)\/rspathAtt\-\[topology\/(?P<pod>pod\-\d)\/paths\-(?P<nodes>\d*)\/pathep\-\[(?P<pg>eth\d\S*)\]\]')
    
    # vPC
    p2 = re.compile(r'^uni\/tn\-(?P<tenant>\S*)\/ap-(?P<ap>\S*)\/epg\-(?P<epg>\S*)\/rspathAtt\-\[topology\/(?P<pod>pod\-\d)\/protpaths\-(?P<nodes>\d*\-\d*)\/pathep\-\[(?P<pg>\S*)\]\]')
    
    # Port channel
    
    p3 = re.compile(r'^uni\/tn\-(?P<tenant>\S*)\/ap-(?P<ap>\S*)\/epg\-(?P<epg>\S*)\/rspathAtt\-\[topology\/(?P<pod>pod\-\d)\/paths\-(?P<nodes>\d*)\/pathep\-\[(?P<pg>\S*)\]\]')
    

    
    m1 = p1.match(dn)
    m2 = p2.match(dn)
    m3 = p3.match(dn)
    
    if m1:
        
        group = m1.groupdict()
        epg = group['epg']
        pod = group['pod']
        ap = group['ap']
        nodes  =  group['nodes']
        
        tenant = group['tenant']

        pg = group['pg']

        ptype= 'access'
   
    
    
    # vPC dn

    
    elif m2:
        group = m2.groupdict()
        epg = group['epg']
        pod = group['pod']
        ap = group['ap']
        nodes  =  group['nodes']
        
        tenant = group['tenant']

        pg = group['pg']

        ptype= 'vPC'


    # PC dn
    
    
    elif m3:
        group = m3.groupdict()
        ptype = 'PC'
        
        epg = group['epg']
        pod = group['pod']
        ap = group['ap']
        nodes  =  group['nodes']
        
        tenant = group['tenant']

        pg = group['pg']


    epg_df = epg_df.append({'tenant': tenant,'ap': ap,'epg': epg,'pod': pod,'nodes': nodes,'type': ptype,'policy_group': pg,'encap': encap},ignore_index=True)

sum2 = pd.merge(sum1, switch_profile_df, on='switch_profile',how='left')


uni/infra/funcprof/accbundle-VPC_01_IPG
uni/infra/funcprof/accbundle-VPC_01_IPG
uni/infra/funcprof/accbundle-PC_01_IPG
uni/infra/funcprof/accportgrp-aci_p03_intpolg_access
uni/infra/funcprof/accportgrp-aci_p03_intpolg_access
uni/infra/funcprof/accportgrp-aci_p03_intpolg_access
uni/infra/funcprof/accportgrp-aci_p03_intpolg_access


In [216]:
switch_profile_df

,switch_profile,fromPort,toPort
0,vpc2,21,21
1,vpc_leaf_interface_profle,18,18
2,pc_leaf_interface_profile,12,16
3,CX_int_profile4,1,10
4,CX_int_profile2,1,10
5,CX_int_profile,1,10
6,CX_int_profile3,1,10


In [217]:
policy_df

,switch_profile,policy_group
0,vpc2,VPC_01_IPG
1,vpc_leaf_interface_profle,VPC_01_IPG
2,pc_leaf_interface_profile,PC_01_IPG
3,CX_int_profile4,aci_p03_intpolg_access
4,CX_int_profile2,aci_p03_intpolg_access
5,CX_int_profile,aci_p03_intpolg_access
6,CX_int_profile3,aci_p03_intpolg_access


In [218]:
epg_df

,tenant,ap,epg,pod,nodes,type,policy_group,encap
0,CX,CX_ap,EPG100,pod-1,101,access,eth1/2,vlan-100
1,CX,CX_ap,EPG100,pod-1,101,PC,PC_01_IPG,vlan-222
2,CX,CX_ap,EPG100,pod-1,101,access,eth1/3,vlan-111
3,CX,CX_ap,EPG300,pod-1,102,access,eth1/2,vlan-300
4,CX,CX_ap,EPG300,pod-1,101-102,vPC,VPC_01_IPG,vlan-333


In [219]:
sum1 = pd.merge(epg_df, policy_df, on='policy_group',how='left')


In [220]:
sum1

,tenant,ap,epg,pod,nodes,type,policy_group,encap,switch_profile
0,CX,CX_ap,EPG100,pod-1,101,access,eth1/2,vlan-100,NaN
1,CX,CX_ap,EPG100,pod-1,101,PC,PC_01_IPG,vlan-222,pc_leaf_interface_profile
2,CX,CX_ap,EPG100,pod-1,101,access,eth1/3,vlan-111,NaN
3,CX,CX_ap,EPG300,pod-1,102,access,eth1/2,vlan-300,NaN
4,CX,CX_ap,EPG300,pod-1,101-102,vPC,VPC_01_IPG,vlan-333,vpc2
5,CX,CX_ap,EPG300,pod-1,101-102,vPC,VPC_01_IPG,vlan-333,vpc_leaf_interface_profle


In [221]:
sum2 = pd.merge(sum1, switch_profile_df, on='switch_profile',how='left')


In [222]:
sum2

,tenant,ap,epg,pod,nodes,type,policy_group,encap,switch_profile,fromPort,toPort
0,CX,CX_ap,EPG100,pod-1,101,access,eth1/2,vlan-100,NaN,NaN,NaN
1,CX,CX_ap,EPG100,pod-1,101,PC,PC_01_IPG,vlan-222,pc_leaf_interface_profile,12,16
2,CX,CX_ap,EPG100,pod-1,101,access,eth1/3,vlan-111,NaN,NaN,NaN
3,CX,CX_ap,EPG300,pod-1,102,access,eth1/2,vlan-300,NaN,NaN,NaN
4,CX,CX_ap,EPG300,pod-1,101-102,vPC,VPC_01_IPG,vlan-333,vpc2,21,21
5,CX,CX_ap,EPG300,pod-1,101-102,vPC,VPC_01_IPG,vlan-333,vpc_leaf_interface_profle,18,18


In [212]:
sum2.to_csv('epg_map.csv')